<a href="https://colab.research.google.com/github/Tincho1902/PosgradoIA/blob/main/1a_vectorizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [159]:
import numpy as np

In [160]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [161]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [162]:
# Transformo cada documento en una lista de términos

corpus_list = np.char.split(corpus)
print(corpus_list)

# Vector con las palabras presentes en mi corpus
univ_palab = []
for sublist in corpus_list:
    univ_palab.extend(sublist)
univ_palab = np.unique(univ_palab)
print(univ_palab)


[list(['que', 'dia', 'es', 'hoy'])
 list(['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'])
 list(['martes', 'muchas', 'gracias'])]
['de' 'dia' 'el' 'es' 'gracias' 'hoy' 'martes' 'muchas' 'que']


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [163]:
# Creo un diccionario
diccionario = {}
for index, palabra in enumerate(sorted(list(univ_palab))):
  diccionario[palabra] = index

# Luego genero los vectores de cada documento
vector = np.zeros((len(corpus_list),len(univ_palab)), dtype=int)
for idx, doc in enumerate(corpus):
  for palabra in corpus_list[idx]:
    col = diccionario.get(palabra)
    vector[idx, col] += 1
    #print('Palabra:', palabra, '\t| Pos:',col,'| Idx:', idx,'\n', vector)
#print(univ_palab,'\n', corpus_list)
print(vector)

[[0 1 0 1 0 1 0 0 1]
 [1 1 1 1 0 1 2 0 0]
 [0 0 0 0 1 0 1 1 0]]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [164]:
np.set_printoptions(precision=3)

In [165]:
# Cuento aparicion de cada palabra
frec = np.zeros((len(univ_palab)), dtype=int)
for idx, doc in enumerate(corpus):
  for palabra in np.unique(corpus_list[idx]):
    col = diccionario.get(palabra)
    frec[col] += 1
#print(frec)

# Calculo el log(N/DF)
N = np.ones((len(univ_palab)), dtype=int)*len(corpus_list)
DF = frec
idf = np.log(N/DF)
print(idf)

[1.099 0.405 1.099 0.405 1.099 0.405 0.405 1.099 1.099]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [166]:
print(vector*idf)

[[0.    0.405 0.    0.405 0.    0.405 0.    0.    1.099]
 [1.099 0.405 1.099 0.405 0.    0.405 0.811 0.    0.   ]
 [0.    0.    0.    0.    1.099 0.    0.405 1.099 0.   ]]


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [201]:
def comparacion_documentos(corpus, ind):
  # Transformo cada documento en una lista de términos
  corpus_list = np.char.split(corpus)

  # Vector con las palabras presentes en mi corpus
  univ_palab = []
  for sublist in corpus_list:
      univ_palab.extend(sublist)
  univ_palab = np.unique(univ_palab)
  # Creo un diccionario
  diccionario = {}
  for index, palabra in enumerate(sorted(list(univ_palab))):
    diccionario[palabra] = index
  # Luego genero los vectores de cada documento
  vector = np.zeros((len(corpus_list),len(univ_palab)), dtype=int)
  for idx, doc in enumerate(corpus):
    for palabra in corpus_list[idx]:
      col = diccionario.get(palabra)
      vector[idx, col] += 1
  frec = np.zeros((len(univ_palab)), dtype=int)
  for idx, doc in enumerate(corpus):
    for palabra in np.unique(corpus_list[idx]):
      col = diccionario.get(palabra)
      frec[col] += 1
  # Calculo el log(N/DF)
  N = np.ones((len(univ_palab)), dtype=int)*len(corpus_list)
  DF = frec
  idf = np.log(N/DF)
  # Calculo la matriz TF-IDF
  matriz_tfidf = vector*idf
  # Busco la similitud coseno del documento seleccionado con el index y el resto del corpus
  similitud = np.zeros(len(corpus))
  for i in range(len(corpus)):
    similitud[i] = cosine_similarity(matriz_tfidf[ind], matriz_tfidf[i])
  # Ordeno segun similitud
  indices_ordenados = np.argsort(similitud)[::-1]
  # Devuelve los documentos ordenados por similitud coseno
  documentos_ordenados = [(corpus[i], similitud[i]) for i in indices_ordenados]

  return documentos_ordenados

In [211]:
corpus_2 = np.array(['el sol brilla sobre las hojas del otoño',
                     'un gato duerme',
                     'mi perro duerme',
                     'el perro es blanco',
                     'nubes en el cielo',
                     'hojas caen al suelo',
                     'el viento sopla sobre el suelo',
                     'mis gatos juegan con mi perro',
                     'los perros corren',
                     'perros y gatos juegan'
          ])

In [213]:
comparacion_documentos(corpus_2,9)

[('perros y gatos juegan', 1.0000000000000002),
 ('mis gatos juegan con mi perro', 0.32180784028705395),
 ('los perros corren', 0.1972309157563855),
 ('el viento sopla sobre el suelo', 0.0),
 ('hojas caen al suelo', 0.0),
 ('nubes en el cielo', 0.0),
 ('el perro es blanco', 0.0),
 ('mi perro duerme', 0.0),
 ('un gato duerme', 0.0),
 ('el sol brilla sobre las hojas del otoño', 0.0)]